In [3]:
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_community.llms import GigaChat
from pydantic import BaseModel, Field
from typing import Optional

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [ ]:
# Определяем структуру вывода с помощью Pydantic
class RentalInfo(BaseModel):
    total_persons: int = Field(description="Общее количество проживающих")

# Настройка GigaChat
llm = GigaChat(
    credentials="YzJiZGZmM2MtYjdlMy00OGE5LWE5YmItNTg3MDZiYjJiZTE3OmY2OTc2NmJlLWU2NmQtNDBhYS1iNjVhLWFkZDQxNmNkYmFiMA==", 
    verify_ssl_certs=False,
    model='GigaChat-2',
    temperature=0.1
)


In [9]:
# Создаем парсер вывода
parser = PydanticOutputParser(pydantic_object=RentalInfo)

# Создаем промпт-шаблон
prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты - AI-ассистент для извлечения структурированной информации из заявок на аренду жилья. 
Извлекай только общее количество проживающих (total_persons) из текста заявки.
Верни ответ строго в формате JSON."""),
    ("human", "Текст заявки: {text}\n\n{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())


In [14]:
# Создаем цепочку
chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)
# Загрузка данных
applications = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/01/rental_applications_text.csv", sep = ";")
true_values = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/01/rental_applications_amount.csv", sep = ",")

results = []
correct_count = 0

for idx, row in applications.iterrows():
    try:
        # Получаем предсказание
        result = chain.invoke({"text": row["text"]})
        predicted = result["text"].total_persons
        
        # Сравниваем с истинным значением
        true_val = true_values[true_values["text_id"] == row["text_id"]]["amount"].values[0]
        is_correct = predicted == true_val
        
        if is_correct:
            correct_count += 1
            
        results.append({
            "text_id": row["text_id"],
            "predicted_total_persons": predicted,
            "true_total_persons": true_val,
            "correct": is_correct
        })
        
    except Exception as e:
        print(f"Ошибка при обработке text_id {row['text_id']}: {str(e)}")
        results.append({
            "text_id": row["text_id"],
            "predicted_total_persons": None,
            "true_total_persons": true_val,
            "correct": False
        })

# Сохраняем результаты
results_df = pd.DataFrame(results)
results_df.to_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/01/results.csv", index=False)

# Вычисляем точность
accuracy = correct_count / len(applications)
print(f"Точность: {accuracy:.2%}")

Ошибка при обработке text_id 237679797: Failed to parse RentalInfo from completion {"total_persons": null}. Got: 1 validation error for RentalInfo
total_persons
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Ошибка при обработке text_id 102311098: Failed to parse RentalInfo from completion {"total_persons": null}. Got: 1 validation error for RentalInfo
total_persons
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/int_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Ошибка при обработке text_id 54495881: Failed to parse RentalInfo from completion {"total_persons": null}. Got: 1 validation e

In [32]:
class SafePydanticOutputParser(PydanticOutputParser):
    def parse(self, text: str) -> BaseModel:
        try:
            # Пытаемся распарсить JSON
            data = json.loads(text)
            
            # Заменяем None на 0 для числовых полей
            for field in ['count_adults', 'count_children']:
                if field in data and data[field] is None:
                    data[field] = 0
            
            # Создаем объект Pydantic
            return self.pydantic_object.parse_obj(data)
        except json.JSONDecodeError:
            # Если это не JSON, пытаемся извлечь числа из текста
            numbers = re.findall(r'\d+', text)
            data = {
                "total_persons": int(numbers[0]) if numbers else 0,
                "count_adults": int(numbers[1]) if len(numbers) > 1 else 0,
                "count_children": int(numbers[2]) if len(numbers) > 2 else 0
            }
            return self.pydantic_object.parse_obj(data)


In [37]:

class RentalInfo(BaseModel):
    total_persons: int = Field(description="Общее количество проживающих", default =0)
    count_adults: int = Field(description="Количество взрослых", default =0)
    count_children: int = Field(description="Количество детей", default =0)


# Создаем парсер вывода
parser = PydanticOutputParser(pydantic_object=RentalInfo)
#parser = SafePydanticOutputParser(pydantic_object=RentalInfo)
# Создаем промпт-шаблон для дополнительной задачи
additional_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты - AI-ассистент для извлечения структурированной информации из заявок на аренду жилья. 
Извлекай следующую информацию из текста заявки:
1. Общее количество проживающих (total_persons)
2. Количество взрослых (count_adults)
3. Количество детей (count_children)
Учти, что общее количество проживающих равняется сумме взрослых и детей.
Если какая-то информация отсутствует, верни 0 для этого поля.
Верни ответ строго в формате JSON."""),
    ("human", "Текст заявки: {text}\n\n{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())

# Создаем цепочку для дополнительной задачи
additional_chain = LLMChain(llm=llm, prompt=additional_prompt, output_parser=parser)



In [40]:
additional_results = []
correct_adults = 0
correct_children = 0
correct_both = 0
additional_true = pd.read_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/01/additional_true.csv", sep = ",")
# Ограничиваем обработку первыми 20 записями из additional_true.csv
additional_true_subset = additional_true.head(20)

for idx, row in additional_true_subset.iterrows():
    try:
        # Находим соответствующий текст заявки
        application_row = applications[applications["text_id"] == row["text_id"]]
        if application_row.empty:
            print(f"Не найден текст для text_id: {row['text_id']}")
            continue
            
        text = application_row.iloc[0]["text"]
        
        # Получаем предсказание для дополнительной задачи
        result = additional_chain.invoke({"text": text})
        predicted = result["text"]
        if(predicted.count_adults) is None:
            predicted.count_adults = 0
        if(predicted.count_children) is None:
            predicted.count_children = 0
        if(predicted.total_persons) is None:
            predicted.total_persons = 0
        # Сравниваем с истинными значениями
        true_val_adults = additional_true[additional_true["text_id"] == row["text_id"]]["adults_count"].values[0]
        adults_correct = predicted.count_adults == true_val_adults
        true_val_children = additional_true[additional_true["text_id"] == row["text_id"]]["children_count"].values[0]
        children_correct = predicted.count_children == true_val_children
        true_both = additional_true[additional_true["text_id"] == row["text_id"]]["total_count"].values[0]
        #both_correct = predicted.count_total == true_both
        # both_correct = adults_correct and children_correct
        
        # Обновляем счетчики точности
        if adults_correct:
            correct_adults += 1
        if children_correct:
            correct_children += 1
        #if both_correct:
        #    correct_both += 1
            
        # Сохраняем результат
        additional_data = {
            "text_id": row["text_id"],
            "text": text,
            "predicted_total_persons": predicted.total_persons,
            "true_total_count": row["total_count"],
            "total_correct": predicted.total_persons == row["total_count"],
            "predicted_count_adults": predicted.count_adults,
            "true_adults_count": row["adults_count"],
            "adults_correct": adults_correct,
            "predicted_count_children": predicted.count_children,
            "true_children_count": row["children_count"],
            "children_correct": children_correct,
            "both_correct": both_correct
        }
        
        additional_results.append(additional_data)
        print(f"Обработана запись {idx+1}/{len(additional_true_subset)}: {row['text_id']}")
        
    except Exception as e:
        print(f"Ошибка при обработке text_id {row['text_id']} для дополнительной задачи: {str(e)}")
        additional_results.append({
            "text_id": row["text_id"],
            "text": None,
            "predicted_total_persons": None,
            "true_total_count": row["total_count"],
            "total_correct": False,
            "predicted_count_adults": None,
            "true_adults_count": row["adults_count"],
            "adults_correct": False,
            "predicted_count_children": None,
            "true_children_count": row["children_count"],
            "children_correct": False,
            "both_correct": False
        })

# Сохраняем результаты дополнительной задачи
additional_df = pd.DataFrame(additional_results)
additional_df.to_csv("D:/maga_poly/ML_3_sem/ml_polytech_3_sem/01/additional_results.csv", index=False)

n = len(additional_true_subset)
if n > 0:
    accuracy_adults = correct_adults / n
    accuracy_children = correct_children / n
    #accuracy_both = correct_both / n
else:
    accuracy_adults = 0
    accuracy_children = 0
    accuracy_both = 0

print(f"Точность дополнительной задачи (взрослые): {accuracy_adults:.2%}")
print(f"Точность дополнительной задачи (дети): {accuracy_children:.2%}")
#print(f"Точность дополнительной задачи (оба поля): {accuracy_both:.2%}")

Обработана запись 1/20: 14205200
Обработана запись 2/20: 319097075
Обработана запись 3/20: 98881311
Обработана запись 4/20: 44587027
Обработана запись 5/20: 352802829
Обработана запись 6/20: 52157605
Обработана запись 7/20: 408430986
Обработана запись 8/20: 176294447
Обработана запись 9/20: 150296790
Обработана запись 10/20: 120627580
Обработана запись 11/20: 447606306
Обработана запись 12/20: 6705832
Обработана запись 13/20: 98410800
Обработана запись 14/20: 89356440
Обработана запись 15/20: 8267186
Ошибка при обработке text_id 12546114 для дополнительной задачи: Invalid json output: ```json
{
  "total_persons": 4,
  "count_adults": 0,  // Информация отсутствует, принимаем значение по умолчанию
  "count_children": 0 // Информация отсутствует, принимаем значение по умолчанию
}
```  

**Пояснение:**  
Заявка содержит указание на общее количество проживающих — 4 человека. Однако конкретные данные о количестве взрослых и детей отсутствуют, поэтому возвращаем значения по умолчанию (0).
For


# Создаем отчет в Jupyter Notebook формате
report_content = f"""
# Отчет по практической работе №1

## Основная задача: Извлечение общего количества проживающих

### Методология
Использовалась библиотека LangChain с моделью GigaChat для извлечения структурированной информации.
Был создан Pydantic-класс для определения структуры вывода и соответствующий промпт.

### Точность
Общая точность на всем датасете: {accuracy:.2%}

### Примеры извлеченных данных (первые 5 записей):
"""

print(report_content)
print(additional_df.head().to_string())

# Дополнительно: сохраняем отчет в файл
with open("report.md", "w", encoding="utf-8") as f:
    f.write(report_content)
    f.write("\n```\n")
    f.write(additional_df.head().to_string())
    f.write("\n```\n")